In [15]:
import gspread

gc = gspread.oauth()

In [16]:
wb_deepwork = gc.open('Deep Work')
wb_tracker = gc.open('Personal Tracking')
sh_focus = wb_deepwork.worksheet('Focus')
cells_focus = sh_focus.get_all_records()

In [17]:
def get_cells(gc, workbook_name, worksheet_name):
    workbook = gc.open(workbook_name)
    worksheet = workbook.worksheet(worksheet_name)
    cells = worksheet.get_all_records()

    return cells

In [18]:
cells_focus = get_cells(gc, workbook_name='Deep Work', worksheet_name='Focus')

In [19]:
import pandas as pd, numpy as np
from quant_helper.data_cleaning import cols_to_datetime, format_datetime_index

df_focus = (
    pd.DataFrame(cells_focus)
    .pipe(cols_to_datetime, cols=['Start', 'Stop'])
    .assign(duration = lambda x: (x['Stop'] - x['Start']).divide(pd.Timedelta(hours=1)))
    .dropna(axis=0, subset=['Type'])
)

In [20]:
df_focus.tail()

,Task,Start,Stop,Type,Quality,Notes,Distractions,Date,Hours,Checks/Hr,MA(10),,duration
169,Ken's Training,2020-07-14 12:13:04,2020-07-14 13:01:00,Education,5,Screwed around the whole time,5,,,,,,0.798889
170,Constrained Equipment Models Meeting,2020-07-14 13:01:00,2020-07-14 13:41:09,Meeting,6,,5,,,,,,0.669167
171,Create Strategy Report Template,2020-07-14 13:51:24,2020-07-14 14:12:00,Deep,10,,0,,,,,,0.343333
172,Fix ETHUSD Data,2020-07-14 14:12:00,2020-07-14 15:39:00,Deep,6,Bathroom Break,5,,,,,,1.450000
173,Fix ETHUSD Data,2020-07-14 16:01:00,2020-07-14 16:54:32,Deep,,,2,,,,,,0.892222


In [21]:
import matplotlib.pyplot as plt

summary_focus_daily = (
    df_focus
    .groupby([df_focus['Start'].dt.date, 'Type'])
    ['duration']
    .sum()
    .unstack()
    .pipe(format_datetime_index)
    .resample('D')
    .last()
    .fillna(0)
)

In [22]:
summary_focus_daily

Type,Deep,Education,Meeting,None,Shallow
date,,,,,
2020-01-09,3.152222,0.000000,0.000000,0.0,0.000000
2020-01-10,5.115278,0.000000,0.000000,0.0,0.000000
2020-01-11,0.000000,0.000000,0.000000,0.0,0.000000
2020-01-12,4.779722,0.000000,0.000000,0.0,0.000000
2020-01-13,2.497500,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...
2020-07-10,1.275833,0.000000,0.750000,0.0,2.562500
2020-07-11,4.231667,0.000000,0.000000,0.0,1.591944
2020-07-12,0.000000,0.000000,0.000000,0.0,0.000000


In [23]:
# def convert_col_dtypes(df, cols, dtypes):
#     for col, dtype in zip(cols, dtypes):
#         df[col] = df[col].astype(dtype)

#     return df

from quant_helper.data_cleaning import convert_col_dtypes

In [24]:
summary_focus_daily_formatted = (
    summary_focus_daily
    .reset_index()
    .pipe(convert_col_dtypes, ['date'], [str])
)

In [25]:
summary_focus_daily_formatted

Type,date,Deep,Education,Meeting,None,Shallow
0,2020-01-09,3.152222,0.000000,0.000000,0.0,0.000000
1,2020-01-10,5.115278,0.000000,0.000000,0.0,0.000000
2,2020-01-11,0.000000,0.000000,0.000000,0.0,0.000000
3,2020-01-12,4.779722,0.000000,0.000000,0.0,0.000000
4,2020-01-13,2.497500,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...
183,2020-07-10,1.275833,0.000000,0.750000,0.0,2.562500
184,2020-07-11,4.231667,0.000000,0.000000,0.0,1.591944
185,2020-07-12,0.000000,0.000000,0.000000,0.0,0.000000
186,2020-07-13,4.933333,2.424722,0.000000,0.0,1.988611


In [26]:
def update_gsheet_pandas(worksheet, df):
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [27]:
sh_work = wb_tracker.worksheet('Work')

In [28]:
update_gsheet_pandas(sh_work, summary_focus_daily_formatted)

# Logging Stuff

In [29]:
from discord_webhook import DiscordWebhook, DiscordEmbed